Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [4]:
all_data.sample(3)
# Target: monthly sum

,shop_id,item_id,date_block_num,target
6453076,41,3039,17,0.0
6571333,31,8575,18,0.0
9130743,54,6316,26,0.0


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [5]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


#### Method 2

In [6]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [4]:
grader = Grader()

train_new = all_data.copy()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [5]:
# YOUR CODE GOES HERE
from sklearn.model_selection import KFold

kf = KFold(5, shuffle=False)

#train_new = all_data.copy()
train_new['item_target_enc_KFold'] = np.nan

for tr_ind, val_ind in kf.split(train_new):
    X_tr, X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    item_id_target_mean = X_tr.groupby('item_id').target.mean()
    X_val['item_target_enc_KFold'] = X_val['item_id'].map(item_id_target_mean)
    train_new.iloc[val_ind] = X_val

# Fill NaNs
train_new['item_target_enc_KFold'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = train_new['item_target_enc_KFold'].values
    
# You will need to compute correlation like that
corr = np.corrcoef(train_new['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

# CORRECT: Current answer for task KFold_scheme is: 0.41645907128

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


0.41645907128
Current answer for task KFold_scheme is: 0.41645907128


In [11]:
#train_new.head(5)

# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [6]:
# YOUR CODE GOES HERE
# LEAVE ONE OUT -- ONE ITEM id?
gr = train_new.groupby("item_id")
item_id_sums = gr.target.sum()
item_id_sizes = gr.target.size()
#print(item_id_sums.head(3))

LOO_sum = train_new['item_id'].map(item_id_sums)
LOO_size = train_new['item_id'].map(item_id_sizes)

# Leave one row out...
# for that category the sum minus this one per the size without this row...
train_new['item_target_enc_LOO'] = (LOO_sum - train_new["target"]) / (LOO_size - 1)

# Fill NaNs
train_new['item_target_enc_LOO'].fillna(0.3343, inplace=True) 

encoded_feature = train_new['item_target_enc_LOO'].values

corr = np.corrcoef(train_new['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

# CORRECT: Current answer for task Leave-one-out_scheme is: 0.480384831129

item_id
0    1.0
1    6.0
2    2.0
Name: target, dtype: float64
139255     1.0
141495    42.0
144968    84.0
Name: item_id, dtype: float64
0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


In [15]:
train_new.head(10)  # id = item_id --> alignaa

,shop_id,item_id,date_block_num,target,item_target_enc_KFold,item_target_enc_LOO,item_target_enc_Smooth,item_target_enc_expMean
139255,0,19,0,0.0,0.334300,0.022727,0.237448,0.3343
141495,0,27,0,0.0,0.048523,0.056911,0.089905,0.3343
144968,0,28,0,0.0,0.142424,0.141414,0.168964,0.3343
142661,0,29,0,0.0,0.030303,0.037500,0.107910,0.3343
138947,0,32,0,6.0,0.894020,1.316088,1.260635,0.3343
138948,0,33,0,3.0,0.485238,0.525552,0.515676,0.3343
138949,0,34,0,0.0,0.107018,0.146283,0.166235,0.3343
139247,0,35,0,1.0,0.020408,0.944444,0.762478,0.3343
142672,0,40,0,0.0,0.054717,0.070997,0.089425,0.3343
142065,0,41,0,0.0,0.038136,0.086000,0.127171,0.3343


In [15]:
train_new.head(5)

,shop_id,item_id,date_block_num,target,item_target_enc_KFold,item_target_enc_LOO,item_target_enc_Smooth
139255,0,19,0,0.0,0.334300,0.334273,0.237448
141495,0,27,0,0.0,0.048523,0.334273,0.089905
144968,0,28,0,0.0,0.142424,0.334273,0.168964
142661,0,29,0,0.0,0.030303,0.334273,0.107910
138947,0,32,0,6.0,0.894020,0.334273,1.260635


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [7]:
# YOUR CODE GOES HERE
alpha = 100
globalmean = 0.3343

# Without CV loop(?)
gr = train_new.groupby("item_id")
item_id_target_mean = gr.target.mean()
item_id_target_size = gr.target.size()
item_id_enc = (item_id_target_mean * item_id_target_size + globalmean * alpha) / (item_id_target_size + alpha)

train_new['item_target_enc_Smooth'] = train_new['item_id'].map(item_id_enc)

train_new['item_target_enc_Smooth'].fillna(0.3343, inplace=True) 

encoded_feature = train_new['item_target_enc_Smooth'].values

corr = np.corrcoef(train_new['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

# CORRECT:  Current answer for task Smoothing_scheme is: 0.48181987971

0.48181987971
Current answer for task Smoothing_scheme is: 0.48181987971


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [8]:
# YOUR CODE GOES HERE
cumsum = train_new.groupby("item_id")["target"].cumsum() - train_new["target"] 
cumcnt = train_new.groupby("item_id").cumcount()

train_new['item_target_enc_expMean'] = cumsum / cumcnt
train_new['item_target_enc_expMean'].fillna(0.3343, inplace=True)

encoded_feature = train_new['item_target_enc_expMean'].values

corr = np.corrcoef(train_new['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

# CORRECT: Current answer for task Expanding_mean_scheme is: 0.502524521108

0.502524521108
Current answer for task Expanding_mean_scheme is: 0.502524521108


In [17]:
print(train_new.groupby("item_id")["target"].cumsum().shape)   # item_id as id
print( train_new["target"].shape)
print(cumsum.shape)

(10913850,)
(10913850,)
(10913850,)


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [11]:
STUDENT_EMAIL = ""# EMAIL HERE
STUDENT_TOKEN = ""# TOKEN HERE
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645907128
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.48181987971
Task Expanding_mean_scheme: 0.502524521108


In [12]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
